In [436]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [437]:
import numpy as np
import pandas as pd
import json
import utils
import itertools

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [438]:
config_dicts = utils.get_config_dicts()

In [439]:
pd.DataFrame(config_dicts).sort_values(["dataset", "base_model"])

,folder,base_model,scoring_model,mask_model,dataset,sampling,d_path,z_path
0,EleutherAI-gpt-neo-125m_EleutherAI-gpt-neo-125...,EleutherAI-gpt-neo-125m,EleutherAI-gpt-neo-125m,t5-base,german,temp,results/permute/EleutherAI-gpt-neo-125m_Eleuth...,results/permute/EleutherAI-gpt-neo-125m_Eleuth...
2,EleutherAI-gpt-neo-125m_bert-base-cased_t5-bas...,EleutherAI-gpt-neo-125m,bert-base-cased,t5-base,german,temp,results/permute/EleutherAI-gpt-neo-125m_bert-b...,results/permute/EleutherAI-gpt-neo-125m_bert-b...
4,EleutherAI-gpt-neo-125m_gpt2-medium_t5-base_ge...,EleutherAI-gpt-neo-125m,gpt2-medium,t5-base,german,temp,results/permute/EleutherAI-gpt-neo-125m_gpt2-m...,results/permute/EleutherAI-gpt-neo-125m_gpt2-m...
6,EleutherAI-gpt-neo-125m_gpt2_t5-base_german_temp,EleutherAI-gpt-neo-125m,gpt2,t5-base,german,temp,results/permute/EleutherAI-gpt-neo-125m_gpt2_t...,results/permute/EleutherAI-gpt-neo-125m_gpt2_t...
8,EleutherAI-gpt-neo-125m_roberta-base_t5-base_g...,EleutherAI-gpt-neo-125m,roberta-base,t5-base,german,temp,results/permute/EleutherAI-gpt-neo-125m_robert...,results/permute/EleutherAI-gpt-neo-125m_robert...
20,gpt2_EleutherAI-gpt-neo-125m_t5-base_german_temp,gpt2,EleutherAI-gpt-neo-125m,t5-base,german,temp,results/permute/gpt2_EleutherAI-gpt-neo-125m_t...,results/permute/gpt2_EleutherAI-gpt-neo-125m_t...
22,gpt2_bert-base-cased_t5-base_german_temp,gpt2,bert-base-cased,t5-base,german,temp,results/permute/gpt2_bert-base-cased_t5-base_g...,results/permute/gpt2_bert-base-cased_t5-base_g...
24,gpt2_gpt2-medium_t5-base_german_temp,gpt2,gpt2-medium,t5-base,german,temp,results/permute/gpt2_gpt2-medium_t5-base_germa...,results/permute/gpt2_gpt2-medium_t5-base_germa...
26,gpt2_gpt2_t5-base_german_temp,gpt2,gpt2,t5-base,german,temp,results/permute/gpt2_gpt2_t5-base_german_temp/...,results/permute/gpt2_gpt2_t5-base_german_temp/...
28,gpt2_roberta-base_t5-base_german_temp,gpt2,roberta-base,t5-base,german,temp,results/permute/gpt2_roberta-base_t5-base_germ...,results/permute/gpt2_roberta-base_t5-base_germ...


In [440]:
config_df = pd.DataFrame(config_dicts)

In [441]:
base_models = sorted(set(config_df["base_model"].values))
datasets = sorted(set(config_df["dataset"].values))
drop_scoring_base_bools = [False, True]

In [442]:
ensemble_configs = list(dict(zip(("base_model", "drop_scoring_base", "dataset"), p)) for p in itertools.product(base_models, drop_scoring_base_bools, datasets))

In [443]:
def get_aucs(**kwargs):
    df = utils.get_pred_df(**kwargs)
    
    y = df["y"]
    X = df.drop("y", axis=1).values

    auc_dict = {}

    auc_dict["max"] = roc_auc_score(y, X.max(axis=1))
    auc_dict["mean"] =  roc_auc_score(y, X.mean(axis=1))
    auc_dict["median"] = roc_auc_score(y, np.median(X, axis=1))

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model_lr = LogisticRegression()
    model_lr.fit(X_train, y_train)
    auc_dict["lr"] = roc_auc_score(y_test, model_lr.predict_proba(X_test)[:, 1])

    model_rf = RandomForestClassifier()
    model_rf.fit(X_train, y_train)
    auc_dict["rf"] = roc_auc_score(y_test, model_rf.predict_proba(X_test)[:, 1])
    return auc_dict

In [444]:
df_ensemble_results_nested = pd.DataFrame([{**c, "aucs":get_aucs(**c)} for c in ensemble_configs])

In [445]:
df_ensemble_results = df_ensemble_results_nested.join(df_ensemble_results_nested["aucs"].apply(pd.Series)).drop("aucs", axis=1)

In [446]:
utils.get_auc_single().sort_values(["base_model", "scoring_model"])

,base_model,scoring_model,dataset,auc
0,EleutherAI-gpt-neo-125m,EleutherAI-gpt-neo-125m,german,0.953675
1,EleutherAI-gpt-neo-125m,EleutherAI-gpt-neo-125m,xsum,0.998200
2,EleutherAI-gpt-neo-125m,bert-base-cased,german,0.369700
3,EleutherAI-gpt-neo-125m,bert-base-cased,xsum,0.628175
6,EleutherAI-gpt-neo-125m,gpt2,german,0.823425
7,EleutherAI-gpt-neo-125m,gpt2,xsum,0.696050
4,EleutherAI-gpt-neo-125m,gpt2-medium,german,0.775925
5,EleutherAI-gpt-neo-125m,gpt2-medium,xsum,0.247875
8,EleutherAI-gpt-neo-125m,roberta-base,german,0.465750
9,EleutherAI-gpt-neo-125m,roberta-base,xsum,0.840350


In [447]:
df_ensemble_results

,base_model,drop_scoring_base,dataset,max,mean,median,lr,rf
0,EleutherAI-gpt-neo-125m,False,german,0.948550,0.891050,0.780375,0.968349,0.978365
1,EleutherAI-gpt-neo-125m,False,xsum,0.783300,0.922950,0.846525,1.000000,1.000000
2,EleutherAI-gpt-neo-125m,True,german,0.633600,0.730225,0.765425,0.767045,0.756494
3,EleutherAI-gpt-neo-125m,True,xsum,0.318825,0.721425,0.739750,0.975160,0.956931
4,gpt2,False,german,0.911950,0.825625,0.719425,1.000000,1.000000
5,gpt2,False,xsum,0.817600,0.946425,0.933750,0.998397,0.998397
6,gpt2,True,german,0.545200,0.434125,0.452700,0.968169,0.953286
7,gpt2,True,xsum,0.557825,0.884550,0.910900,0.995986,0.989964
8,gpt2-medium,False,german,0.930225,0.912325,0.811975,0.999599,0.996387
9,gpt2-medium,False,xsum,0.925200,0.956850,0.906950,0.945978,0.930372
